In [18]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [19]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Bambous Virieux,20,MU,1/14/2021 17:40,78,-20.3428,57.7575,78.80,3.44
1,1,Katsuura,0,JP,1/14/2021 17:40,53,35.1333,140.3000,48.20,16.11
2,2,Butaritari,63,KI,1/14/2021 17:40,79,3.0707,172.7902,80.83,23.55
3,3,Tuktoyaktuk,90,CA,1/14/2021 17:40,84,69.4541,-133.0374,-7.60,6.91
4,4,Krasnoarmeysk,100,RU,1/14/2021 17:40,83,51.0239,45.6969,17.01,3.00


In [20]:
city_data_df.dtypes

City_ID         int64
City           object
Cloudiness      int64
Country        object
Date           object
Humidity        int64
Lat           float64
Lng           float64
Max Temp      float64
Wind Speed    float64
dtype: object

In [21]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [22]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [29]:

# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [30]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [39]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [40]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Bambous Virieux,20,MU,1/14/2021 17:40,78,-20.3428,57.7575,78.80,3.44
2,2,Butaritari,63,KI,1/14/2021 17:40,79,3.0707,172.7902,80.83,23.55
5,5,Hermanus,0,ZA,1/14/2021 17:40,73,-34.4187,19.2345,79.00,5.99
6,6,Castro,51,BR,1/14/2021 17:40,68,-24.7911,-50.0119,76.84,11.32
7,7,Rikitea,42,PF,1/14/2021 17:40,76,-23.1203,-134.9692,77.22,11.07
11,11,Cairns,40,AU,1/14/2021 17:40,78,-16.9167,145.7667,78.80,9.22
14,14,Vaini,75,TO,1/14/2021 17:40,100,-21.2000,-175.2000,75.20,4.61
15,15,Gondanglegi,100,ID,1/14/2021 17:40,85,-7.7479,112.0132,75.22,3.18
27,27,Avarua,90,CK,1/14/2021 17:35,83,-21.2078,-159.7750,78.80,6.91
36,36,Kibala,0,TD,1/14/2021 17:40,29,9.1104,18.3463,87.80,4.07


In [41]:
preferred_cities_df.count()

City_ID       164
City          164
Cloudiness    164
Country       163
Date          164
Humidity      164
Lat           164
Lng           164
Max Temp      164
Wind Speed    164
dtype: int64

In [42]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Bambous Virieux,MU,78.80,-20.3428,57.7575,
2,Butaritari,KI,80.83,3.0707,172.7902,
5,Hermanus,ZA,79.00,-34.4187,19.2345,
6,Castro,BR,76.84,-24.7911,-50.0119,
7,Rikitea,PF,77.22,-23.1203,-134.9692,
11,Cairns,AU,78.80,-16.9167,145.7667,
14,Vaini,TO,75.20,-21.2000,-175.2000,
15,Gondanglegi,ID,75.22,-7.7479,112.0132,
27,Avarua,CK,78.80,-21.2078,-159.7750,
36,Kibala,TD,87.80,9.1104,18.3463,


In [43]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [48]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
   # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
      
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [49]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [51]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [52]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))